In [ ]:
import sys, os, importlib, numpy as np, pandas as pd
sys.path.append(os.path.abspath(".."))

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

from src import data as data_mod, features as features_mod, utils as utils_mod
importlib.reload(data_mod); importlib.reload(features_mod); importlib.reload(utils_mod)
from src.data import get_data
from src.features import add_features
from src.utils import make_labels

# Optional XGBoost
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except Exception:
    HAS_XGB = False

plt.rcParams["figure.figsize"] = (12,5)
plt.rcParams["axes.grid"] = True


In [ ]:
TICKERS = ["AAPL", "SPY", "MSFT", "BTC-USD"]  # edit freely
START, END = "2015-01-01", "2023-12-31"

TAUS = [0.0, 0.001, 0.002]   # 0%, ±0.1%, ±0.2% dead-zone
FEE = 0.0010                 # 10 bps one-way
MODELS = ["logreg", "xgb"]   # will skip xgb if not installed
RANDOM_STATE = 42

In [ ]:
def split_scale(X, y, r, train=0.70, val=0.15):
    n = len(X); i_tr = int(train*n); i_va = int((train+val)*n)
    X_tr, y_tr, r_tr = X[:i_tr], y[:i_tr], r[:i_tr]
    X_va, y_va, r_va = X[i_tr:i_va], y[i_tr:i_va], r[i_tr:i_va]
    X_te, y_te, r_te = X[i_va:], y[i_va:], r[i_va:]
    scaler = StandardScaler().fit(X_tr)
    return (scaler.transform(X_tr), y_tr, r_tr,
            scaler.transform(X_va), y_va, r_va,
            scaler.transform(X_te), y_te, r_te)

def sharpe_daily(rr):
    rr = pd.Series(rr); s = rr.std()
    return 0.0 if s==0 else rr.mean()/s*np.sqrt(252)

def backtest_from_probs(p, r_next, thr, fee):
    pos = (p > thr).astype(int)
    trades = np.abs(np.diff(np.r_[0, pos])) * fee
    strat_r = pos * r_next - trades
    return {
        "Sharpe_strat": sharpe_daily(strat_r),
        "Sharpe_bh": sharpe_daily(r_next),
        "Coverage": pos.mean(),
        "Trades": int(trades.sum()),
    }

def tune_threshold(p_val, r_val, fee=FEE, grid=np.linspace(0.50, 0.60, 21)):
    # choose thr that maximizes fee-aware Sharpe on validation
    best = None
    best_s = -1e9
    for t in grid:
        s = backtest_from_probs(p_val, r_val, thr=t, fee=fee)["Sharpe_strat"]
        if s > best_s:
            best_s, best = s, t
    return best


In [ ]:
def run_once(ticker, tau):
    # 1) data
    df = get_data(ticker, start=START, end=END)
    df = add_features(df)
    df = make_labels(df, tau=tau, dead_zone=(tau > 0))
    feat_cols = [c for c in df.columns if c not in ["date","open","high","low","close","volume","ret_next","y"]]
    X = df[feat_cols].values
    y = df["y"].astype(int).values
    r = df["ret_next"].values

    # 2) split & scale
    X_tr, y_tr, r_tr, X_va, y_va, r_va, X_te, y_te, r_te = split_scale(X, y, r)

    rows = []

    # 3) Logistic Regression
    lr = LogisticRegression(max_iter=2000, class_weight="balanced", random_state=RANDOM_STATE)
    lr.fit(X_tr, y_tr)
    p_va = lr.predict_proba(X_va)[:,1]
    p_te = lr.predict_proba(X_te)[:,1]
    thr = tune_threshold(p_va, r_va, fee=FEE)
    bt = backtest_from_probs(p_te, r_te, thr, fee=FEE)
    rows.append(dict(
        ticker=ticker, tau=tau, model="logreg",
        auc_va=roc_auc_score(y_va, p_va),
        auc_te=roc_auc_score(y_te, p_te),
        thr=thr, **bt
    ))

    # 4) XGBoost (if available)
    if HAS_XGB and "xgb" in MODELS:
        xgb = XGBClassifier(
            n_estimators=400, max_depth=4, learning_rate=0.05,
            subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
            random_state=RANDOM_STATE, n_jobs=-1
        )
        xgb.fit(X_tr, y_tr)
        p_va_x = xgb.predict_proba(X_va)[:,1]
        p_te_x = xgb.predict_proba(X_te)[:,1]
        thr_x = tune_threshold(p_va_x, r_va, fee=FEE)
        bt_x = backtest_from_probs(p_te_x, r_te, thr_x, fee=FEE)
        rows.append(dict(
            ticker=ticker, tau=tau, model="xgb",
            auc_va=roc_auc_score(y_va, p_va_x),
            auc_te=roc_auc_score(y_te, p_te_x),
            thr=thr_x, **bt_x
        ))

    return pd.DataFrame(rows)


In [ ]:
all_rows = []
for t in TICKERS:
    for tau in TAUS:
        try:
            res = run_once(t, tau)
            all_rows.append(res)
        except Exception as e:
            print(f"Error on {t}, tau={tau}: {e}")

mt = pd.concat(all_rows, ignore_index=True)
display(mt.sort_values(["ticker","model","tau"]))

print("\n=== Averages by model (across tickers & taus) ===")
display(mt.groupby("model")[["auc_te","Sharpe_strat","Sharpe_bh","Coverage","Trades"]].mean().round(4))

print("\n=== Best per ticker/model by Sharpe_strat ===")
best = (mt.sort_values(["ticker","model","Sharpe_strat"], ascending=[True,True,False])
          .groupby(["ticker","model"]).head(1)
          .reset_index(drop=True))
display(best[["ticker","model","tau","auc_te","Sharpe_strat","Sharpe_bh","Coverage","Trades","thr"]])


In [ ]:
def fee_sweep(ticker="AAPL", tau=0.001, fees=(0.0005, 0.0010, 0.0020)):
    out = []
    base_fee = FEE
    for f in fees:
        globals()["FEE"] = f
        df = run_once(ticker, tau)
        df["fee"] = f
        out.append(df)
    globals()["FEE"] = base_fee
    return pd.concat(out, ignore_index=True)

fee_res = fee_sweep("AAPL", 0.001)
display(fee_res[["model","fee","auc_te","Sharpe_strat","Sharpe_bh","Coverage","Trades","thr"]].sort_values(["model","fee"]))


In [ ]:
from pathlib import Path
Path("../data").mkdir(parents=True, exist_ok=True)

mt.to_csv("../data/multiticker_tau_sweep.csv", index=False)
best.to_csv("../data/multiticker_best_by_ticker_model.csv", index=False)
fee_res.to_csv("../data/multiticker_fee_sweep_AAPL.csv", index=False)

"Saved: data/multiticker_tau_sweep.csv, data/multiticker_best_by_ticker_model.csv, data/multiticker_fee_sweep_AAPL.csv"
